In [70]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import pymongo
import re
import pymysql
from sqlalchemy import create_engine
from config import gkey
pymysql.install_as_MySQLdb()
import time

In [71]:
# getting the info for sql bench
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [79]:
# Creating engine to mysql
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [80]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# San Juan 18.466333,-66.105721
# Ponce 18.01108, -66.61406
# Rincon 18.340216, -67.250015
# Fajardo 18.325787, -65.652382
# Arecibo 18.47245, -66.71573

# spanish, puertorican, middle eastern, chinese, fast food, bar restaurant
params = {
    "location": "18.01108, -66.61406",  # San Juan coords
    "radius": 32000,
    "types": "restaurant",
    "key": gkey,
    "keyword": ""
}

response = requests.get(base_url, params=params).json()
#response

In [81]:
#results[8]['photos'][0]['photo_reference']

In [82]:
def generate_results_dataframe(results):
    results_list = []

    for r in results:
        test = True

        # Initialize variables
        business_status = None
        lat = None
        lng = None
        icon = None,
        name = None
        place_id = None
        price_level = None
        photo_reference = None
        rating = None
        place_types = None
        user_ratings_total = None
        address = None

        try:
            if 'business_status' in r.keys(): business_status = r['business_status']
        except: pass

        try:
            if 'lat' in r['geometry']['location'].keys(): lat = r['geometry']['location']['lat']
        except: pass
        try:
            if 'lng' in r['geometry']['location'].keys(): lng = r['geometry']['location']['lng']
        except: pass

        try:
            if 'icon' in r.keys(): icon = r['icon']
        except: pass

        try:
            if 'name' in r.keys(): name = r['name']
        except: pass

        try:
            if 'place_id' in r.keys(): place_id = r['place_id']
        except: pass

        try:
            if 'price_level' in r.keys(): price_level = r['price_level']
        except: pass

        try:
            if 'photos' in r.keys(): photo_reference = r['photos'][0]['photo_reference']
        except: pass

        try:
            if 'rating' in r.keys(): rating = r['rating']
        except: pass

        try:
            if 'types' in r.keys(): place_types = r['types']
        except: pass

        try:
            if 'user_ratings_total' in r.keys(): user_ratings_total = r['user_ratings_total']
        except: pass

        try:
            if 'vicinity' in r.keys(): address = r['vicinity']
        except: pass

        result_dict = {
            'business_status': business_status,
            'lat': lat,
            'lng': lng,
            'icon': icon,
            'name': name,
            'place_id': place_id,
            'price_level': price_level,
            'photo_reference': photo_reference,
            'rating': rating,
            'place_types': place_types,
            'user_ratings_total': user_ratings_total,
            'address': address
        }

        results_list.append(result_dict)

    results_df = pd.DataFrame(results_list)
    results_df['place_types'] = results_df['place_types'].astype(str)
    
    return results_df

In [83]:
while('results' in response.keys() and len(response['results']) > 0):   
        
    results = response['results']
    results_df = generate_results_dataframe(results)
    results_df.to_sql('restaurants', con=conn, if_exists='append', index=False)
    print(f'--- {len(results_df)} record(s) loaded to the database --- ')
    
    if 'next_page_token' in response:      
        pagetoken = response['next_page_token']
    
        # sleeping for the "short delay" 
        time.sleep(10)
    
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
            "pagetoken": pagetoken,
            "key": gkey,
        }
    
        response = requests.get(base_url, params=params).json()
        
    else:
        break

--- 20(s) loaded to the database --- 
ATtYBwKPW-IvPDlgc1paIOH1PsmAGDu-6h9LnVo9J0AOHzq0J9UUGVcqRJjzHFSZdDOM5gon6t2LrQp6VcJ5utK6bIbYWI0-WdUbwTGxvUZYxW7ZwY_cy_V8BQ-_QncZXU2SiDEhqYSWhAywxiPtR9wgBv2JIoaJp03IRaSsMYyCDDn-l6sgHLQOSsqta1S2TjXqQwymE-DpCdDDG-Q2c6yV9z7W90Ku2gOzGjOqZ8xuZAoGEunnCp-Wyp2g0wVaDwYFUMsAVkFASFR6kjONEGfvxnEB6q9c87zqDad6I_EsQYbHKvlHaKBVS_Ok2coxHZa7Ld6dR6YT2PoY8IHmpd53m82OJiXWbs3bO7vG70G8ILrbxeWf5cCGmR_Opcds7c3J_sxnPsMHCb0_1feWkXDAHAcRIqeomlV3Ugw-Erf6yQUpjvRJfyb696z_jKRJ
--- 20(s) loaded to the database --- 
ATtYBwK3_iO7awsSi5phigxgfsyC_mzV9myBsx7faGV3nUHrZn7SG3Mrc2Sk3lEfeTxHQin6xk6uUxJyLZRPeLBp0hTcln0-m-hkuU11rqWxyzfQe5md8l5TkSS-Mq9UwS-tg1B28zBkdpNPWcMYXU-wrhklTyi5eUt8pYYkL5ThSj2L413SpKNTDyH8Z_xRvKsUtW0-MN0in1PvDYf0hyQ7SFBaqQizy3hYDjAu7u1VAxyxkDY2BaHtMiRPDXTCUH7a-3Mbc2FkfN2DA1jpDSTfgCM8s9-G-7f7e7-Mdq8WPC4Ke7luiKcaAYJAGkUDsc3E1PDt5iR_EjWHRH9efUUznghXLOkoM8RVH5rvfdDcbo-g7SBM5Lno4h06tNObl9Hqi_lDdAiTymmDPco-knuvnrXP5Xs821bwRskr2rzvAwqvCQRZJpmAWgyt_smP7NyHW5bB3XXPm78D5olYP791EC1

In [10]:
response

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# San Juan 18.466333,-66.105721
# Ponce 18.01108, -66.61406
# Rincon 18.340216, -67.250015
# Fajardo 18.325787, -65.652382
# Arecibo 18.47245, -66.71573

# spanish, puertorican, middle eastern, chinese, fast food, bar restaurant
params = {
    "location": "18.01108,-66.61406",  # Ponce coords
    "rankby": "distance",
    "type": "restaurant",
    "key": gkey,
    "keyword": "spanish"
}

response = requests.get(base_url, params=params).json()
response

results = response['results']
results

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# San Juan 18.466333,-66.105721
# Ponce 18.01108, -66.61406
# Rincon 18.340216, -67.250015
# Fajardo 18.325787, -65.652382
# Arecibo 18.47245, -66.71573

# spanish, puertorican, middle eastern, chinese, fast food, bar restaurant
params = {
    "location": "18.340216,-67.250015",  # Rincon coords
    "rankby": "distance",
    "type": "restaurant",
    "key": gkey,
    "keyword": "spanish"
}

response = requests.get(base_url, params=params).json()
response

results = response['results']
results

In [ ]:
connection = pymysql.connect(host='databootcamp.cbyaoibag9dz.us-east-2.rds.amazonaws.com',
                             user='admin',
                             password='2013Parc!',
                             db='pr')

In [ ]:
cursor = connection.cursor()

In [ ]:
sql = "INSERT INTO `restaurants` (`restaurantname`, `rating`, `address`, `url`) VALUES (%s, %s, %s, %s)"

# Execute the query
cursor.execute(sql, ('Han Cream', 3.7,'201 Ca. Tanca, San Juan', 'https://maps.google.com/maps/contrib/100796978647196078569'))

In [ ]:
# for row in results: 
#     text = row["photos"][0]["html_attributions"][0]
#     url = re.findall(r'"(.*?)"', text)
    
#     info = f"""
#     Name:  {row["name"]}
#     URL:  {url[0]}
#     Rating:  {row["rating"]}
#     Address:  {row["vicinity"]}
#     """

for row in results:
    try:
        Name =  row["name"]
        #URL =  row["photos"]["html_attributions"]
        Rating =  row["rating"]
        Address =  row["vicinity"]
        Status = row["business_status"]
     
        print('------------')
        print(Name)
        print(Rating)
        print(Address)
        print(Status)
        #print(URL)
        
    except KeyError as e:
        print(e)
    
# # # create code that will insert into a sql data.    
# #     dic = {
# #         'Name': row["name"],
# #         'Type': "Chinese",
# #         'Address': row['vicinity']
# #     }
    
#     print(info)    